In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10, mnist
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.layers import MaxPooling2D
import numpy as np
import scipy
import matplotlib
import h5py
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

np.random.seed(0)

In [2]:
#Code from StackExchange to fix 
import sys 
from PIL import Image
sys.modules['Image'] = Image 

In [3]:
#Initialize the CNN
size = 64
cnn_layers = [Conv2D(64, (3, 3), input_shape=(size,size,3), padding='same', activation='relu'),
Conv2D(64, (3, 3), activation='relu', padding='same'),
MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
Conv2D(128, (3, 3), activation='relu', padding='same'),
Conv2D(128, (3, 3), activation='relu', padding='same',),
MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
Conv2D(256, (3, 3), activation='relu', padding='same',),
Conv2D(256, (3, 3), activation='relu', padding='same',),
Conv2D(256, (3, 3), activation='relu', padding='same',),
MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
Conv2D(512, (3, 3), activation='relu', padding='same',),
Conv2D(512, (3, 3), activation='relu', padding='same',),
Conv2D(512, (3, 3), activation='relu', padding='same',),
MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
Conv2D(512, (3, 3), activation='relu', padding='same',),
Conv2D(512, (3, 3), activation='relu', padding='same',),
Conv2D(512, (3, 3), activation='relu', padding='same',),
MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
Flatten(),
Dense(4096, activation='relu'),
Dense(4096, activation='relu'),
Dense(1, activation='sigmoid')
]



classifier = keras.Sequential(cnn_layers)

classifier.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [4]:
#Used to artificially increase the size of the training data set

#Blog.fkeras code for creating batches of randomly transformed images
#img = load_img('Z:\School\CSCI470\deepfake_detect\Images\Training\Faked\Copy of 000000.png')  # this is a PIL image
#x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
#x = x.reshape((1,) + x.shape)

#i = 0
#for batch in datagen.flow(x, batch_size=1,
#                          save_to_dir='preview', save_prefix='Faked', save_format='jpeg'):
#    i += 1
#    if i > 20:
#        break

In [5]:
train_datagen = ImageDataGenerator(rescale = 1./255,
shear_range = 0.2,
zoom_range = 0.2,
horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

training_path = 'Z:\\School\\CSCI470\\deepfake_detect\\HardData2\\Train'
test_path = 'Z:\\School\\CSCI470\\deepfake_detect\\HardData2\\Validation'

training_set = train_datagen.flow_from_directory(training_path,
target_size = (size,size),
batch_size = 25,
class_mode = 'binary')

test_set = test_datagen.flow_from_directory(test_path,
target_size = (size,size),
batch_size = 25,
class_mode = 'binary')



#Make sure labels are applied correctly
print(training_set.labels)

Found 4000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
[0 0 0 ... 1 1 1]


In [6]:
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')

hist = classifier.fit_generator(steps_per_epoch=100,generator=training_set, validation_data= test_set, validation_steps=10,epochs=20,callbacks=[checkpoint,early])

Epoch 1/20
100/100 [==============================] - 343s 3s/step - loss: 0.7119 - acc: 0.5076 - val_loss: 0.6933 - val_acc: 0.4920
Epoch 2/20
100/100 [==============================] - 388s 4s/step - loss: 0.6932 - acc: 0.5096 - val_loss: 0.6933 - val_acc: 0.4920
Epoch 3/20
100/100 [==============================] - 350s 3s/step - loss: 0.6932 - acc: 0.5016 - val_loss: 0.6932 - val_acc: 0.4920
Epoch 4/20
100/100 [==============================] - 305s 3s/step - loss: 0.6932 - acc: 0.4972 - val_loss: 0.6931 - val_acc: 0.5080
Epoch 5/20
  7/100 [=>............................] - ETA: 2:52 - loss: 0.6931 - acc: 0.5314

In [0]:
# test the NN on a single image
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('gdrive/My Drive/AppliedProject/NewData/Test/Fake/easy_201_0001.jpg', target_size = (size,size))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices
print(result)
if result[0][0] == 1:
  prediction = 'fake'
else:
  prediction = 'real'
  
prediction